# Candidate Evaluation with AISuite
> format: ipynb

> jupyter: python3

# Candidate Evaluation with AISuite

This notebook evaluates job candidates using AISuite by combining system and user prompts 
stored in external files. It retrieves credentials, loads necessary data, 
constructs prompts, and generates AI-driven candidate evaluations.

In [1]:
import os
import json
import toml

from pathlib import Path
from IPython.display import display, Markdown

import pandas as pd

## Configuration and Setup
Define file paths and load credentials.

In [2]:
# Paths configuration
paths = {
    'root': Path.cwd().parent,
    'data': Path.cwd().parent / "data",
    "config": Path.cwd().parent / "config"
}

# Load credentials from config/credentials.json
with open(paths["config"] / 'credentials.json') as f:
    credentials = json.load(f)

Environment variable HUGGINGFACE_TOKEN set.


## Load External Text Prompts
Load system description and user prompt template from external files.

In [3]:
# Load system description and user prompt template from files
with open(paths["config"] / 'role_system_description.md', 'r') as f:
    role_system_description = f.read()

with open(paths["config"] / 'user_prompt_template.md', 'r') as f:
    user_prompt_template = f.read()

with open(paths["config"] / 'response_format.md', 'r') as f:
    response_format = f.read()

with open(paths["config"] / 'search_criteria.md', 'r') as f:
    search_criteria = f.read()


## Load Instructions and Candidate Data

In [ ]:
# Load instructions (if needed for context)
try:
    with open(paths["config"] / "instructions.md", 'r') as file:
        instructions = file.read()
        print("Instructions successfully read!")
except FileNotFoundError:
    instructions = ""
    print("Instructions file not found.")

# Load shortlisted candidates from parquet or CSV
try:
    list_of_candidates = (
        pd.read_parquet(paths['data'] / "processed/filtered.parquet", columns=['job_title'])['job_title']
          .to_list()
    )
except Exception as e:
    print(f"Failed to load parquet file: {e}. Loading CSV instead.")
    list_of_candidates = (
        pd.read_csv(paths['data'] / "processed/filtered.csv", index_col=['rank'], usecols=['rank', 'job_title'])
          ['job_title']
          .to_list()
    )

Instructions successfully read!
Failed to load parquet file: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.. Loading CSV instead.


## Prepare Dynamic Prompt Parameters

In [5]:
# Define dynamic parameters for the prompt
search_terms_path = paths["config"] / 'search_terms.toml'
search_terms = toml.load(search_terms_path)

list_similar_roles = ', '.join(search_terms['search_phrases'])
location = search_terms['location']

# Format the user prompt using the template and parameters
user_prompt = user_prompt_template.format(
    location=location,
    list_similar_roles=list_similar_roles,
    search_criteria=search_criteria,
    list_of_candidates="\n".join(list_of_candidates)  # Joining list elements for display
)

# Combine system description and user prompt for models that don't support system roles
combined_prompt = f"{role_system_description}\n\n{user_prompt}"

## Define AISuite Generation Function


In [11]:
import sys
import importlib.util

# Define the path to the module
module_path = paths['root'] / 'potential_talents' / 'models' / 'aisuite_generate.py'

# Load the module
spec = importlib.util.spec_from_file_location("aisuite_generate", module_path)
aisuite_generate_module = importlib.util.module_from_spec(spec)
sys.modules["aisuite_generate"] = aisuite_generate_module
spec.loader.exec_module(aisuite_generate_module)

# Import the function
aisuite_generate = aisuite_generate_module.aisuite_generate

## Generate and Display Candidate Evaluation

In [12]:
# Prepare messages
messages = [{"role": "user", "content": combined_prompt}]

# Generate text using AISuite
generated_output = aisuite_generate(messages, model="openai:o1-mini", max_tokens=5000)

# Display the generated output in Markdown format
display(Markdown("**Generated Output:**\n" + generated_output))

**Generated Output:**
Here are the top 5 candidates most suitable for an entry-level Human Resources position in New York based on your criteria:

| **Candidate**                                                                                                                                         | **Experience & Background**                                                                                                     | **Reasoning for Choosing This Candidate**                                                                                                                                                      |
|-------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Aspiring Human Resources Professional Passionate About Helping to Create an Inclusive and Engaging Work Environment**                             | Aspiring HR professional with a focus on inclusivity and employee engagement.                                                 | Demonstrates a clear passion for creating positive workplace environments, aligning with company values. Shows motivation and fundamental HR interest, ideal for entry-level roles.              |
| **Retired Army National Guard Recruiter Office Manager Seeking a Position in Human Resources**                                                      | Former Army National Guard recruiter and office manager with transferable leadership and recruitment skills.                   | Brings strong leadership and recruitment experience from the military, indicating high motivation to transition into HR. Aligns well with values emphasizing discipline and teamwork.             |
| **Aspiring Human Resources Manager Seeking Internship in Human Resources**                                                                          | Aspiring HR manager actively seeking internships to gain hands-on experience in the field.                                     | Proactively seeking opportunities to enter the HR field, showcasing strong motivation and commitment. Ideal for entry-level positions requiring eagerness to learn and grow.                      |
| **Aspiring Human Resources Professional: An Energetic and Team-Focused Leader**                                                                       | Energetic, team-focused leader with aspirations in human resources.                                                           | Exhibits strong interpersonal skills and enthusiasm for HR, essential for collaborative and dynamic HR environments. Balances motivation with promising leadership potential.                    |
| **CT Bauer College of Business Graduate, Magna Cum Laude, and Aspiring Human Resources Professional**                                               | Recent business graduate with high academic achievements and a focus on human resources.                                       | Exceptional academic background combined with a clear focus on HR demonstrates both capability and motivation. Prepared to apply solid foundational knowledge to entry-level HR roles.           |